# Back to Instance Properties - Coding

In [1]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

In [1]:
import weakref

class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

In [2]:
class Point:
    x = IntegerValue()
    
    def __init__(self, x):
        self.x = x
        
p = Point(1)
p.__dict__
p.x

1

In [17]:
p = Point()
print(hex(id(p)))

0x22e7e1100a0


In [18]:
p.x = 100.1

In [19]:
p.x

100

In [20]:
Point.x.values.keyrefs()

[<weakref at 0x0000022E7DC06EF0; to 'Point' at 0x0000022E7E1100A0>]

In [21]:
del p

In [22]:
Point.x.values.keyrefs()

[]

In [4]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(id(instance))
        

In [5]:
class Point:
    x = IntegerValue()
    
    def __init__(self, x):
        self.x = x
        
    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x

In [6]:
p = Point(10.1)

In [7]:
p.__dict__

{}

In [8]:
p.x

10

In [9]:
p.x = 20.2

In [10]:
p.x

20

In [11]:
id(p), Point.x.values

(140310390235824, {140310390235824: 20})

In [12]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [13]:
p_id = id(p)
ref_count(p_id)

1

In [14]:
del p
ref_count(p_id)

1

In [15]:
Point.x.values

{140310390235824: 20}

In [16]:
p = Point(10.1)
weak_p = weakref.ref(p)
ref_count(id(p))
del p
weak_p

<weakref at 0x7f9cac0f2f90; dead>

In [21]:
def obj_destroyed(obj):
    print(f"{obj} is being destroyed")

In [22]:
p = Point(10.1)
w = weakref.ref(p, obj_destroyed)

In [23]:
del p

<weakref at 0x7f9cac06d720; dead> is being destroyed


In [30]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object)
                                     ,int(value))
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(id(instance))[1]
        
    def _remove_object(self, weak_ref):
        print(f"removing dead entry for {weak_ref}")

In [31]:
class Point:
    x = IntegerValue()

In [32]:
p1 = Point()
p2 = Point()

p1.x, p2.x = 10.1, 100.1
p1.x, p2.x

(10, 100)

In [33]:
ref_count(id(p1)), ref_count(id(p2))

(1, 1)

In [34]:
del p1

removing dead entry for <weakref at 0x7f9c8e2089a0; dead>


In [35]:
Point.x.values

{140310371935808: (<weakref at 0x7f9c8e2089a0; dead>, 10),
 140310371936960: (<weakref at 0x7f9c8e2082c0; to 'Point' at 0x7f9c8de0aac0>,
  100)}

In [37]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object)
                                     ,int(value))
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(id(instance))[1]
        
    def _remove_object(self, weak_ref):
        print(f"removing dead entry for {weak_ref}")
#         reverse_lookup = [key for key, value in self.values.items()
#                          if value[0] is weak_ref]
#         if reverse_lookup:
#             key = reverse_lookup[0]
#             del self.values[key]
            
        for key, value in self.values.items():
            if value[0] is weak_ref:
                del self.values[key]
                break

In [38]:
class Point:
    x = IntegerValue()

In [39]:
p1 = Point()
p2 = Point()

p1.x, p2.x = 10.1, 100.1
p1.x, p2.x

(10, 100)

In [40]:
del p1

removing dead entry for <weakref at 0x7f9c8dc16e50; dead>


In [41]:
Point.x.values

{140310376246720: (<weakref at 0x7f9c8df4f450; to 'Point' at 0x7f9c8e226dc0>,
  100)}

In [42]:
class Person:
    pass

Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [44]:
hasattr(Person.__weakref__, "__set__"), hasattr(Person.__weakref__, "__get__")

(True, True)

In [45]:
p = Person()
hasattr(p, "__weakref__")

True

In [46]:
print(p.__weakref__)

None


In [47]:
w = weakref.ref(p)
print(p.__weakref__)

<weakref at 0x7f9c8e0dbe50; to 'Person' at 0x7f9c8df236d0>


In [48]:
class Person:
    __slots__ = "name",
    
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name',),
              'name': <member 'name' of 'Person' objects>,
              '__doc__': None})

In [49]:
p = Person()
hasattr(p, "__weakref__")

False

In [50]:
w = weakref.ref(p)

TypeError: cannot create weak reference to 'Person' object

In [51]:
class Person:
    __slots__ = "name", "__weakref__"
    
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name', '__weakref__'),
              'name': <member 'name' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [53]:
p = Person()
hasattr(p, "__weakref__")
w = weakref.ref(p)

In [86]:
class ValidString:
    def __init__(self, min_length=0, max_length=255):
        self.data = {}
        self._min_length = min_length
        self._max_length = max_length
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError("Value must be a string")
        if len(value) < self._min_length:
            raise ValueError(
            f"Value should be at least {self._min_length} characters."                
            )
        if len(value) > self._max_length:
            raise ValueError(
            f"Value cannot exceed {self._max_length} characters."                
            )
        self.data[id(instance)] = (weakref.ref(instance, self._finalize_instance), value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]
    
    def _finalize_instance(self, weak_ref):
        print(f"removing dead entry for {weak_ref}")
        reverse_lookup = [key for key, value in self.data.items()
                         if value[0] is weak_ref]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.data[key]

In [87]:
class Person:
    __slots__ = "__weakref__",
    
    first_name = ValidString(1, 100)
    last_name = ValidString(1, 100)
    
    def __eq__(self, other):
        return (
            isinstance(other, Person) and
            self.first_name == other.first_name and
            self.last_name == other.last_name
        )
    
class BankAccount:
    __slots__ = "__weakref__",
    
    account_number = ValidString(5, 255)
    
    def __eq__(self, other):
        return (isinstance(other, BankAccount) and
               self.account_number == other.account_number)
    

In [88]:
p1 = Person()
p2 = Person()
p1.first_name, p1.last_name = "Guido", "van Rossum"
p2.first_name, p2.last_name = "Raymond", "Hettinger"

b1, b2 = BankAccount(), BankAccount()
b1.account_number, b2.account_number = "Savings", "Checking"
p1.first_name, p1.last_name, p2.first_name, p2.last_name

('Guido', 'van Rossum', 'Raymond', 'Hettinger')

In [89]:
b1.account_number, b2.account_number

('Savings', 'Checking')

In [90]:
Person.first_name.data

{140310374211008: (<weakref at 0x7f9c8d6af9a0; to 'Person' at 0x7f9c8e035dc0>,
  'Guido'),
 140310371212112: (<weakref at 0x7f9c8d6b83b0; to 'Person' at 0x7f9c8dd59b50>,
  'Raymond')}

In [91]:
del p1
del p2
del b1
del b2

removing dead entry for <weakref at 0x7f9c8d6b8130; dead>
removing dead entry for <weakref at 0x7f9c8d6af9a0; dead>
removing dead entry for <weakref at 0x7f9c8d6b8310; dead>
removing dead entry for <weakref at 0x7f9c8d6b83b0; dead>
removing dead entry for <weakref at 0x7f9c8d6b8270; dead>
removing dead entry for <weakref at 0x7f9c8d6b8220; dead>


In [92]:
Person.first_name.data

{}